ETL 'credits'

In [1]:
# Paso 1: Agregar la ruta de funciones a sys.path
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'code')))

In [ ]:
print(os.path.abspath(os.path.join('..', 'code')) in sys.path)

True


Conversion credits.csv a formato parquet

In [3]:
from CSVtoPARQUET import convertir_csv_a_parquet

df = convertir_csv_a_parquet(
    csv_path='../data/Raw/Movies-20241026T022341Z-001/Movies/credits.csv', 
    parquet_path='../data/processed_data/credits.parquet'    
)

In [4]:
import pandas as pd

df = pd.read_parquet(r'../data/processed_data/credits.parquet')
df.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Desanidar cast

In [5]:
# Verificar la distribución de los tipos de datos en la columna
df['cast'].apply(type).value_counts() 

cast
<class 'str'>    45476
Name: count, dtype: int64

In [6]:
# convertir la columa a una lista de diccionarios
from STRtoLISToDICT import convert_list_dict

df['cast'] = df['cast'].apply(convert_list_dict)

# Mostrar el DataFrame con los tipos corregidos
df[['cast']].head(6)

,cast
0,"[{'cast_id': 14, 'character': 'Woody (voice)',..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah..."
4,"[{'cast_id': 1, 'character': 'George Banks', '..."
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han..."


In [7]:
# verificar cuántos valores en la columna cast son del tipo dict, list, o NoneType
df['cast'].apply(type).value_counts() 

cast
<class 'list'>    45476
Name: count, dtype: int64

In [8]:
from desanidarColumnas import desanidar_columna

# UtilizaR la función definida en el archivo desanidarColumnas que esta en la carpeta code
cast_desanidado = desanidar_columna(df, 'cast', 'cast_desanidado')

cast_desanidado.head(6)

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,0,Hamm (voice),52fe4284c3a36847f8024fa9,2.0,7907.0,John Ratzenberger,5.0,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg


In [9]:
# Agregar la columna id del DataFrame original (df) a cast_desanidado mediante una unión (merge) basada en la columna cast_id
cast_desanidado = cast_desanidado.merge(df[['cast_id', 'id']], on='cast_id', how='left', suffixes=('', '_df'))

In [10]:
# verificar el resultado de la unión 
print(cast_desanidado.head())
print(cast_desanidado.info())

   cast_id                character                 credit_id  gender  \
0        0            Woody (voice)  52fe4284c3a36847f8024f95     2.0   
1        0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99     2.0   
2        0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d     2.0   
3        0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1     2.0   
4        0              Rex (voice)  52fe4284c3a36847f8024fa5     2.0   

        id           name  order                      profile_path  id_df  
0     31.0      Tom Hanks    0.0  /pQFoyx7rp09CJTAb932F2g8Nlho.jpg    862  
1  12898.0      Tim Allen    1.0  /uX2xVf6pMmPepxnvFWyBtjexzgY.jpg    862  
2   7167.0    Don Rickles    2.0  /h5BcaDMPRVLHLDzbQavec4xfSdt.jpg    862  
3  12899.0     Jim Varney    3.0  /eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg    862  
4  12900.0  Wallace Shawn    4.0  /oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg    862  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 9 colum

In [11]:
# Calcular el porcentaje de valores nulos (NaN) en cada columna del DataFrame cast_desanidado
avg_nulos = cast_desanidado.isnull().mean() * 100
avg_nulos

cast_id          0.000000
character        0.428046
credit_id        0.428046
gender           0.428046
id               0.428046
name             0.428046
order            0.428046
profile_path    31.204903
id_df            0.000000
dtype: float64

In [12]:
# Identificar y mostrar las filas en el DataFrame cast_desanidado que tienen valores nulos en columnas específicas
columnas_con_nulos = ['character', 'credit_id', 'gender', 'id', 'name', 'order']
# Identificar las filas nulas.
nulos = cast_desanidado[cast_desanidado[columnas_con_nulos].isnull().any(axis=1)]
nulos

,cast_id,character,credit_id,gender,id,name,order,profile_path,id_df
2274,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124639
3971,240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43475
6666,393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42981
7331,438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24257
10124,595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124472
...,...,...,...,...,...,...,...,...,...
564717,45447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455661
564759,45452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44330
564766,45458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122036
564781,45462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,276895


In [13]:
# Eliminar las filas con valores nulos en el DataFrame cast_desanidado
indices_nulos = nulos.index
cast_desanidado = cast_desanidado.drop(index=indices_nulos)

In [14]:
# Verificar el número de filas después de la eliminación
print(f"Número de filas después de eliminar nulos: {len(cast_desanidado)}")

Número de filas después de eliminar nulos: 562474


Desanidar crew

In [15]:
# Verificar la distribución de los tipos de datos en la columna crew
df['crew'].apply(type).value_counts()

crew
<class 'str'>    45476
Name: count, dtype: int64

In [16]:
# convertir la columa a una lista de diccionarios
from STRtoLISToDICT import convert_list_dict

df['crew'] = df['crew'].apply(convert_list_dict)

# Mostrar el DataFrame con los tipos corregidos
df[['crew']].head(6)

,crew
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,"[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de..."
5,"[{'credit_id': '52fe4292c3a36847f802916d', 'de..."


In [17]:
# verificar cuántos valores en la columna crew son del tipo dict, list, o NoneType
df['crew'].apply(type).value_counts() 

crew
<class 'list'>    45476
Name: count, dtype: int64

In [18]:
from desanidarColumnas import desanidar_columna

# UtilizaR la función definida en el archivo desanidarColumnas que esta en la carpeta code
crew_desanidado = desanidar_columna(df, 'crew', 'crew_desanidado',1)

crew_desanidado

,credit_id,department,gender,id,job,name,profile_path,crew_id
0,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,1
1,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,1
2,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,1
3,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,1
4,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,1
...,...,...,...,...,...,...,...,...
465080,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None,45474
465081,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None,45474
465082,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,45475
465083,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None,45475


In [19]:
# Agregar la columna id del DataFrame original (df) a crew_desanidado mediante una unión (merge) basada en la columna crew_id
crew_desanidado = crew_desanidado.merge(df[['crew_id', 'id']], on='crew_id', how='left', suffixes=('', '_df'))

In [20]:
# verificar el resultado de la unión 
print(crew_desanidado.head())
print(crew_desanidado.info())

                  credit_id department  gender       id         job  \
0  52fe4284c3a36847f8024f49  Directing     2.0   7879.0    Director   
1  52fe4284c3a36847f8024f4f    Writing     2.0  12891.0  Screenplay   
2  52fe4284c3a36847f8024f55    Writing     2.0      7.0  Screenplay   
3  52fe4284c3a36847f8024f5b    Writing     2.0  12892.0  Screenplay   
4  52fe4284c3a36847f8024f61    Writing     0.0  12893.0  Screenplay   

             name                      profile_path  crew_id  id_df  
0   John Lasseter  /7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg        1    862  
1     Joss Whedon  /dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg        1    862  
2  Andrew Stanton  /pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg        1    862  
3      Joel Cohen  /dAubAiZcvKFbboWlj7oXOkZnTSu.jpg        1    862  
4    Alec Sokolow  /v79vlRYi94BZUQnkkyznbGUZLjT.jpg        1    862  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465085 entries, 0 to 465084
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  

In [21]:
# Calcular el porcentaje de valores nulos (NaN) en cada columna del DataFrame crew_desanidado
avg_nulos = crew_desanidado.isnull().mean() * 100
avg_nulos

credit_id        0.165776
department       0.165776
gender           0.165776
id               0.165776
job              0.165776
name             0.165776
profile_path    79.552555
crew_id          0.000000
id_df            0.000000
dtype: float64

In [22]:
# Identificar y mostrar las filas en el DataFrame crew_desanidado que tienen valores nulos en columnas específicas
columnas_con_nulos = ['credit_id', 'department','gender', 'id', 'job', 'name']
# Identificar las filas nulas.
nulos = crew_desanidado[crew_desanidado[columnas_con_nulos].isnull().any(axis=1)]
nulos

,credit_id,department,gender,id,job,name,profile_path,crew_id,id_df
3486,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190,56088
10351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,615,123505
10527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,636,339428
10941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662,318177
11387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,712,365371
...,...,...,...,...,...,...,...,...,...
463769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45237,332543
463954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45262,28469
464056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45278,458618
464444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45349,335251


In [23]:
# Eliminar las filas con valores nulos en el DataFrame cast_desanidado
indices_nulos = nulos.index
crew_desanidado = crew_desanidado.drop(index=indices_nulos)

In [24]:
# Verificar el número de filas después de la eliminación
print(f"Número de filas después de eliminar nulos: {len(crew_desanidado)}")

Número de filas después de eliminar nulos: 464314


Realizar modificaciones para el desarrollo de la API

In [ ]:
# # Preparar el DataFrame para la API
# Convertir los nombres de los actores en la columna name a minúsculas
cast_desanidado['name'] = cast_desanidado['name'].str.lower()
# Convertir los valores de la columna id al tipo de dato int
cast_desanidado['id'] = cast_desanidado['id'].astype(int)

In [26]:
# Eliminamos columnas innecesarias
cast_desanidado = cast_desanidado.drop('profile_path', axis=1)
cast_desanidado = cast_desanidado.drop('order', axis=1)

In [27]:
# Eliminación de la Columna 'character', no se considera relevante
cast_desanidado = cast_desanidado.drop('character', axis=1)

In [ ]:
# Filtrar Directores (no se necesita información de otros roles)
crew_desanidado = crew_desanidado[crew_desanidado['job'] == 'Director']

In [29]:
# Eliminar la Columna 'department'
crew_desanidado = crew_desanidado.drop('department', axis=1)
# Eliminar la Columna 'profile_path'
crew_desanidado = crew_desanidado.drop('profile_path', axis=1)

In [30]:
# Transformar a Minúsculas y Ajustar Tipos de Datos
crew_desanidado['name'] = crew_desanidado['name'].str.lower()
crew_desanidado['id'] = crew_desanidado['id'].astype(int)

In [33]:
# Verificar la estructura de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cast     45476 non-null  object
 1   crew     45476 non-null  object
 2   id       45476 non-null  int64 
 3   cast_id  45476 non-null  int64 
 4   crew_id  45476 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.7+ MB


Load

In [ ]:
# Convertir las listas desanidadas de nuevo a cadenas de texto para facilitar la exportación de los datos
from convertListToString import lista_a_str

# aplicar lista_a_str a cada elemento de las columnas cast y crew
df['cast'] = df['cast'].apply(lista_a_str)
df['crew'] = df['crew'].apply(lista_a_str)

In [36]:
# # Verificar que el directorio de destino exista
output_dir = '../data/processed_data/credits'
os.makedirs(output_dir, exist_ok=True)

Exportacion del DataFrame principal: 
Se crearon las tablas cast_desanidado y crew_desanidado que contienen la información necesaria y están conectadas adecuadamente con el id de movies_dataset,
el DataFrame principal (df) ya no sea necesario

Exportacion de las tablas extras 
(Se exportaran las tablas: cast_desanidado y crew_desanidado)

In [37]:
# Guardar el DataFrame cast_desanidado en un archivo Parquet
# Definir la Ruta y Nombre del Archivo:
parquet_path = os.path.join(output_dir, 'cast_desanidado.parquet')
# Guardar el DataFrame 
cast_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a '../data/processed_data/credits'")

Datos exportados correctamente a '../data/processed_data/credits'


In [40]:
# Cargar y verificar el archivo Parquet
cast_df = pd.read_parquet(parquet_path)
print(cast_df.head())


                  credit_id  gender     id       job             name  \
0  52fe4284c3a36847f8024f49     2.0   7879  Director    john lasseter   
1  52fe44bfc3a36847f80a7c7d     2.0   4945  Director     joe johnston   
2  52fe466a9251416c75077a89     2.0  26502  Director    howard deutch   
3  52fe44779251416c91011acb     2.0   2178  Director  forest whitaker   
4  52fe44959251416c75039eef     2.0  56106  Director    charles shyer   

   crew_id  id_df  
0        1    862  
1        2   8844  
2        3  15602  
3        4  31357  
4        5  11862  


In [38]:
# Guardar el DataFrame crew_desanidado en un archivo Parquet
# Definir la Ruta y Nombre del Archivo:
parquet_path = os.path.join(output_dir, 'crew_desanidado.parquet')
# Guardar el DataFrame
crew_desanidado.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)

print("Datos exportados correctamente a '../data/processed_data/credits'")

Datos exportados correctamente a '../data/processed_data/credits'


In [39]:
# Cargar y verificar el archivo Parquet
crew_df = pd.read_parquet(parquet_path)
print(crew_df.head())


                  credit_id  gender     id       job             name  \
0  52fe4284c3a36847f8024f49     2.0   7879  Director    john lasseter   
1  52fe44bfc3a36847f80a7c7d     2.0   4945  Director     joe johnston   
2  52fe466a9251416c75077a89     2.0  26502  Director    howard deutch   
3  52fe44779251416c91011acb     2.0   2178  Director  forest whitaker   
4  52fe44959251416c75039eef     2.0  56106  Director    charles shyer   

   crew_id  id_df  
0        1    862  
1        2   8844  
2        3  15602  
3        4  31357  
4        5  11862  
